# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import gmaps 
import requests
import os

# Import API key
from config import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saipan,15.1355,145.7010,26.39,83,40,5.14,MP,2024-04-11 15:25:43
1,newman,37.3138,-121.0208,25.58,52,15,0.89,US,2024-04-11 15:25:43
2,hawaiian paradise park,19.5933,-154.9731,22.80,98,100,3.09,US,2024-04-11 15:25:44
3,west island,-12.1568,96.8225,25.99,83,68,3.09,CC,2024-04-11 15:25:44
4,howrah,22.5892,88.3103,27.00,61,40,0.00,IN,2024-04-11 15:25:44


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      tiles = "OSM",
                                      size = "Humidity",
                                      scale = 0.6)

# Display the map
city_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                  & (city_data_df["Max Temp"] > 21)
                                  & (city_data_df["Cloudiness"] ==0)
                                  & (city_data_df["Wind Speed"] < 4.5)]
                                  
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,capacuaro,19.5500,-102.0500,21.65,34,0,0.00,MX,2024-04-11 15:25:46
158,saint-pierre,-21.3393,55.4781,25.82,73,0,2.06,RE,2024-04-11 15:23:08
259,port hedland,-20.3167,118.5667,23.03,88,0,2.06,AU,2024-04-11 15:26:37
290,colares,38.7992,-9.4469,25.55,56,0,3.09,PT,2024-04-11 15:25:57
293,stephenville,32.2207,-98.2023,22.18,37,0,3.09,US,2024-04-11 15:24:08
310,playa del ingles,27.7567,-15.5787,25.86,21,0,3.85,ES,2024-04-11 15:23:09
313,safford,32.8340,-109.7076,26.53,14,0,3.58,US,2024-04-11 15:26:49
319,red bluff,40.1785,-122.2358,25.84,46,0,1.54,US,2024-04-11 15:23:59
443,hurricane,37.1753,-113.2899,26.19,20,0,4.47,US,2024-04-11 15:27:12
493,al bawiti,28.3492,28.8659,21.63,29,0,4.00,EG,2024-04-11 15:27:22


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
def new_func():
    hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()
    return hotel_df

hotel_df = new_func()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,capacuaro,MX,19.5500,-102.0500,
158,saint-pierre,RE,-21.3393,55.4781,
259,port hedland,AU,-20.3167,118.5667,
290,colares,PT,38.7992,-9.4469,
293,stephenville,US,32.2207,-98.2023,
310,playa del ingles,ES,27.7567,-15.5787,
313,safford,US,32.8340,-109.7076,
319,red bluff,US,40.1785,-122.2358,
443,hurricane,US,37.1753,-113.2899,
493,al bawiti,EG,28.3492,28.8659,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
# Increased radius from 10000 metres to 30000 metres to obtain more hotels
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key,
          "limit":20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
def new_func(index):
    hotel_df.loc[index, "Hotel Name"]

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
capacuaro - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
port hedland - nearest hotel: The Esplanade Hotel
colares - nearest hotel: Hotel Arribas
stephenville - nearest hotel: No hotel found
playa del ingles - nearest hotel: Apartamentos Barbados
safford - nearest hotel: Quality Inn And Suites
red bluff - nearest hotel: Travelodge by Wyndham Red Bluff
hurricane - nearest hotel: Wingate by Wyndham Hurricane Near Zion National Park
al bawiti - nearest hotel: Old Oasis Hotel
south hedland - nearest hotel: Port Haven


,City,Country,Lat,Lng,Hotel Name
12,capacuaro,MX,19.5500,-102.0500,No hotel found
158,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
259,port hedland,AU,-20.3167,118.5667,The Esplanade Hotel
290,colares,PT,38.7992,-9.4469,Hotel Arribas
293,stephenville,US,32.2207,-98.2023,No hotel found
310,playa del ingles,ES,27.7567,-15.5787,Apartamentos Barbados
313,safford,US,32.8340,-109.7076,Quality Inn And Suites
319,red bluff,US,40.1785,-122.2358,Travelodge by Wyndham Red Bluff
443,hurricane,US,37.1753,-113.2899,Wingate by Wyndham Hurricane Near Zion Nationa...
493,al bawiti,EG,28.3492,28.8659,Old Oasis Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:

# Configure the map plot

hotel_map_plot = hotel_df.hvplot.points(
            "Lng",
            "Lat",
            geo = True, 
            tiles = "OSM", 
            color = "City",
            frame_width = 600, 
            frame_height = 300,
            hover_cols = ["Hotel Name", "Country"],
            scale = 0.6)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)